# Deep learning framework example: Named Entity Recognition

This notebook demonstrates how to use the deeplearning API to train and test the model on the [CoNNL 2003 dataset](https://www.clips.uantwerpen.be/conll2003/ner/).  The task is to identify named entities (i.e. people, organizations etc).  See the [saved version](https://htmlpreview.github.io/?https://github.com/plandes/deepnlp/blob/master/example/ner/notebook/ner.ipynb) for output.

**Important**: Please see the Movie Review notebook example in the `zensols.movie` API first, as it contains more explaination of how the framework is used.  The purpose of this notebook is to run the MNIST dataset and visualize the results.

In [ ]:
from harness import NotebookHarness
harness = NotebookHarness()
mng = harness()
#mng = harness(1, 'target2')

## Print information about 

Use the factory to create the model executor.  The `write` method gives statistics on the data set that is configured on the executor.

In [ ]:
from zensols.config import Writable
# set indention level for human readable (pretty print) output
Writable.WRITABLE_INDENT_SPACE = 2
facade = mng.create_facade('glove50')
facade.write()

## Train and test the model

Train and test the model with the default (low) number of epochs to make sure everything is working.

In [ ]:
facade.epochs = 2
mng.run()

## Tune hyperparameters

Set model parameters to get a feel for where they need to be before changing features.  Start with Glove 50 dimensional word embeddings with a learning rate of 0.01 and 20 epochs.

In [ ]:
facade.learning_rate = 0.01
facade.epochs = 10
mng.run()

# Glove 300 embeddings

Next we use the same learning rate, but switch to the 300 dimension version of the embeddings.  The number of epochs is reduced because I have run the test before I know at what epoch the validation loss converges.  Since the model is saved only when the validation loss decreases, we early stop at 8 epochs.

In [ ]:
facade.epochs = 8
facade.embedding = 'glove_300_embedding'
mng.run()

# Word2vec Embeddings

Now we switch to the Google 300D word2vec pretrained vectors using 12 epochs, even though it has converged at 9 epochs previously.

In [ ]:
facade.epochs = 12
facade.embedding = 'word2vec_300_embedding'
mng.run()

# BERT Embeddings

Now we test with Bert context aware frozen (not trainable) embeddings using 10 epochs.  We must empty the `net_settings` attributes, which are the lingustic features, since Bert tokenizes using the word piece algorithm and the tensor shapes will not align.  We'll address this later.

In [ ]:
facade = mng.create_facade('transformer-trainable')
mng.run()
facade.persist_result()

In [ ]:
mng.clear()
facade = mng.create_facade('transformer-trainable')
facade.net_settings.add_attributes = ('syns_expander', 'tags_expander')
facade.epochs = 5
mng.run()

In [ ]:
mng.clear()
mng.config('transformer_trainable_resource', model_id='roberta-base')
facade = mng.create_facade('transformer-trainable')
facade.epochs = 8
mng.run()

In [ ]:
import logging
logging.getLogger('zensols.deepnlp.transformer.optimizer').setLevel(logging.INFO)
mng.clear()
mng.config('model_settings',
           scheduler_class_name='zensols.deepnlp.transformer.TransformerSchedulerFactory',
           scheduler_params="dict: {'name': 'linear', 'num_warmup_steps': 0.01}")
facade = mng.create_facade('transformer-trainable')
facade.epochs = 5
mng.run()

In [ ]:
mng.clear()
facade = mng.create_facade('transformer-trainable-large')
mng.run()

In [ ]:
import logging
logging.getLogger('zensols.deepnlp.transformer.optimizer').setLevel(logging.INFO)
mng.clear()
mng.config('model_settings',
           scheduler_class_name='zensols.deepnlp.transformer.TransformerSchedulerFactory',
           scheduler_params="dict: {'name': 'linear', 'num_warmup_steps': 0.01}")
facade = mng.create_facade('transformer-trainable-large')
facade.epochs = 10
mng.run()